In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from importlib import reload 

In [3]:
from deeprank.dataset import DataLoader, PairGenerator, ListGenerator
from deeprank import utils

In [4]:
seed = 1234
torch.manual_seed(seed)

In [5]:
loader = DataLoader('./config/letor07_mp_fold1.model')

[./data/letor/r5w/word_dict.txt]
	Word dict size: 193367
[./data/letor/r5w/qid_query.txt]
	Data size: 1692
[./data/letor/r5w/docid_doc.txt]
	Data size: 65323
[./data/letor/r5w/embed_wiki-pdc_d50_norm]
	Embedding size: 109282
[./data/letor/r5w/embed.idf]
	Embedding size: 193367
Generate numpy embed: (193368, 50)
Generate numpy embed: (193368, 1)


In [6]:
import json
letor_config = json.loads(open('./config/letor07_mp_fold1.model').read())
#device = torch.device("cuda")
#device = torch.device("cpu")
select_device = torch.device("cpu")
rank_device = torch.device("cuda")

In [7]:
Letor07Path = letor_config['data_dir']

letor_config['fill_word'] = loader._PAD_
letor_config['embedding'] = loader.embedding
letor_config['feat_size'] = loader.feat_size
letor_config['vocab_size'] = loader.embedding.shape[0]
letor_config['embed_dim'] = loader.embedding.shape[1]
letor_config['pad_value'] = loader._PAD_

pair_gen = PairGenerator(rel_file=Letor07Path + '/relation.train.fold%d.txt'%(letor_config['fold']), 
                         config=letor_config)

[./data/letor/r5w/relation.train.fold1.txt]
	Instance size: 47828
Pair Instance Count: 325439


In [8]:
from deeprank import select_module
from deeprank import rank_module

In [9]:
letor_config['max_match'] = 20
letor_config['win_size'] = 5
select_net = select_module.QueryCentricNet(config=letor_config, out_device=rank_device)
select_net = select_net.to(select_device)
select_net.train()

QueryCentricNet()

In [10]:
'''
letor_config['q_limit'] = 20
letor_config['d_limit'] = 2000
letor_config['max_match'] = 20
letor_config['win_size'] = 5
letor_config['finetune_embed'] = True
letor_config['lr'] = 0.0001
select_net = select_module.PointerNet(config=letor_config)
select_net = select_net.to(device)
select_net.embedding.weight.data.copy_(torch.from_numpy(loader.embedding))
select_net.train()
select_optimizer = optim.RMSprop(select_net.parameters(), lr=letor_config['lr'])
'''

"\nletor_config['q_limit'] = 20\nletor_config['d_limit'] = 2000\nletor_config['max_match'] = 20\nletor_config['win_size'] = 5\nletor_config['finetune_embed'] = True\nletor_config['lr'] = 0.0001\nselect_net = select_module.PointerNet(config=letor_config)\nselect_net = select_net.to(device)\nselect_net.embedding.weight.data.copy_(torch.from_numpy(loader.embedding))\nselect_net.train()\nselect_optimizer = optim.RMSprop(select_net.parameters(), lr=letor_config['lr'])\n"

In [11]:
letor_config["dim_q"] = 1
letor_config["dim_d"] = 1
letor_config["dim_weight"] = 1
letor_config["c_reduce"] = [1, 1]
letor_config["k_reduce"] = [1, 50]
letor_config["s_reduce"] = 1
letor_config["p_reduce"] = [0, 0]

letor_config["c_en_conv_out"] = 4
letor_config["k_en_conv"] = 3
letor_config["s_en_conv"] = 1
letor_config["p_en_conv"] = 1

letor_config["en_pool_out"] = [1, 1]
letor_config["en_leaky"] = 0.2

letor_config["dim_gru_hidden"] = 3

letor_config['lr'] = 0.005
letor_config['finetune_embed'] = False

rank_net = rank_module.DeepRankNet(config=letor_config)
rank_net = rank_net.to(rank_device)
rank_net.embedding.weight.data.copy_(torch.from_numpy(loader.embedding))
rank_net.qw_embedding.weight.data.copy_(torch.from_numpy(loader.idf_embedding))
rank_net.train()
rank_optimizer = optim.Adam(rank_net.parameters(), lr=letor_config['lr'])

In [12]:
def to_device(*variables, device):
    return (torch.from_numpy(variable).to(device) for variable in variables)

In [13]:
def show_text(x):
    print(' '.join([loader.word_dict[w.item()] for w in x]))

In [14]:
X1, X1_len, X1_id, X2, X2_len, X2_id, Y, F = \
        pair_gen.get_batch(data1=loader.query_data, data2=loader.doc_data)
X1, X1_len, X2, X2_len, Y, F = \
        to_device(X1, X1_len, X2, X2_len, Y, F, device=rank_device)

show_text(X2[0])

X1, X2_new, X1_len, X2_len_new, X2_pos = select_net(X1, X2, X1_len, X2_len, X1_id, X2_id)

show_text(X1[0])
for i in range(5):
    print(i, end=' ')
    show_text(X2_new[0][i])

anniversary whiskeytown dam dedication whiskeytown dam anniversary celebration september visitor help celebrate anniversary president john kennedy dedication whiskeytown dam weekend free special event sept long history seminar turtle bay saturday cover topic multiple perspective water issue recreation overview president kennedy life speech re living exciting moments president person years ago sunday formal ceremony start whiskeytown dam music shasta high school band short speech big dam party follow formal ceremony vendor special interest booth line dam live music provided band billy richards coaster penny lane shadow press release superintendent jim milestone observe good time visitor learned lot national park service whiskeytown local support made happen years ago years ago local support made anniversary celebration happen thank help make event success local support event friends whiskeytown mcconnell foundation turtle bay city red raba bus bureau reclamation forest service bureau la

40th president $$ [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
0 september visitor help celebrate anniversary president john kennedy dedication whiskeytown dam
1 perspective water issue recreation overview president kennedy life speech re living
2 speech re living exciting moments president person years ago sunday formal
3 speech overview anniversary event held president john kennedy dedicated whiskeytown dam
4 

IndexError: index 4 is out of bounds for dimension 0 with size 4

In [15]:
print(X2_pos)

[tensor([ 13.,  38.,  46., 165.], device='cuda:0'), tensor([ 1., 11., 13., 37.], device='cuda:0'), tensor([2.4200e+02, 3.0900e+02, 3.2700e+02, 3.3500e+02, 4.2700e+02, 5.8000e+02,
        1.0820e+03, 1.1360e+03, 1.1660e+03, 1.2480e+03, 1.4300e+03, 1.5600e+03,
        1.5780e+03, 1.8690e+03, 1.0000e+00, 9.0000e+00, 3.1000e+01, 4.3000e+01,
        6.5000e+01, 1.9700e+02, 4.3800e+02, 7.2900e+02, 9.5300e+02, 1.1980e+03,
        1.4460e+03, 1.6940e+03, 1.9490e+03, 9.5600e+02, 1.0320e+03, 1.6220e+03,
        9.0500e+02, 9.0900e+02, 9.2600e+02, 9.5700e+02, 9.7600e+02, 1.0330e+03,
        1.1220e+03], device='cuda:0'), tensor([49., 50.], device='cuda:0'), tensor([ 58., 152.,  59., 153.], device='cuda:0'), tensor([ 4.,  5., 18., 19., 55., 56.], device='cuda:0'), tensor([  3.,  52.,  78.,  87., 111., 143., 193., 199., 260.,   4.,  53.,  79.,
         84.,  88., 102., 112., 115., 144., 180., 194., 200., 233., 261., 286.,
        338., 341., 347., 357., 408.], device='cuda:0'), tensor([ 43.,  59., 

In [16]:
# X1 = X1[:1]
# X1_len = X1_len[:1]
# X2 = X2[:1]
# X2_len = X2_len[:1]
# X1_id = X1_id[:1]
# X2_id = X2_id[:1]

In [17]:
# show_text(X2[0])
# X1, X2_new, X1_len, X2_len_new = select_net(X1, X2, X1_len, X2_len, X1_id, X2_id)
# show_text(X1[0])
# for i in range(5):
#     print(i, end=' ')
#     show_text(X2_new[0][i])

In [18]:
import time
start_t = time.time()
for i in range(1000):
    # One Step Forward
    X1, X1_len, X1_id, X2, X2_len, X2_id, Y, F = \
        pair_gen.get_batch(data1=loader.query_data, data2=loader.doc_data)
    X1, X1_len, X2, X2_len, Y, F = \
        to_device(X1, X1_len, X2, X2_len, Y, F, device=select_device)
    X1, X2, X1_len, X2_len, X2_pos = select_net(X1, X2, X1_len, X2_len, X1_id, X2_id)
    X2, X2_len = utils.data_adaptor(X2, X2_len, select_net, rank_net, letor_config)
    output = rank_net(X1, X2, X1_len, X2_len, X2_pos)
    
    # Update Rank Net
    rank_loss = rank_net.pair_loss(output, Y)
    print('rank loss:', rank_loss.item())
    rank_optimizer.zero_grad()
    rank_loss.backward()
    rank_optimizer.step()
    
end_t = time.time()
print('Time Cost: %s s' % (end_t-start_t))

rank loss: 1.2440992593765259
rank loss: 0.8229913711547852
rank loss: 1.0240232944488525
rank loss: 0.9202936887741089
rank loss: 0.9478692412376404
rank loss: 0.9125843644142151
rank loss: 0.8528660535812378
rank loss: 0.9710915684700012
rank loss: 0.9428439140319824
rank loss: 0.7923462986946106
rank loss: 0.946742057800293
rank loss: 0.6975569128990173
rank loss: 0.7819538116455078
rank loss: 0.8927761912345886
rank loss: 0.7025883197784424
rank loss: 0.8276301622390747
rank loss: 0.7975870370864868
rank loss: 0.7904086709022522
rank loss: 0.7325917482376099
rank loss: 0.7261925935745239
rank loss: 0.7801852226257324
rank loss: 0.6245788335800171
rank loss: 0.7126272320747375
rank loss: 0.7462284564971924
rank loss: 0.7206588387489319
rank loss: 0.7828032374382019
rank loss: 0.7821846008300781
rank loss: 0.6067131757736206
rank loss: 0.6634309887886047
rank loss: 0.6104261875152588
rank loss: 0.9049686789512634
rank loss: 0.638395369052887
rank loss: 0.7226206660270691
rank loss: 0

rank loss: 0.5214572548866272
rank loss: 0.7345959544181824
rank loss: 0.6837531924247742
rank loss: 0.725126326084137
rank loss: 0.572644054889679
rank loss: 0.5535497665405273
rank loss: 0.6656034588813782
rank loss: 0.5773030519485474
rank loss: 0.6440893411636353
rank loss: 0.6690893173217773
rank loss: 0.7227907180786133
rank loss: 0.7116897702217102
rank loss: 0.5992423295974731
rank loss: 0.7305021286010742
rank loss: 0.6894205212593079
rank loss: 0.5510427951812744
rank loss: 0.5643130540847778
rank loss: 0.5811717510223389
rank loss: 0.6286486387252808
rank loss: 0.5052006840705872
rank loss: 0.5410985350608826
rank loss: 0.8264406323432922
rank loss: 0.7780131101608276
rank loss: 0.677638590335846
rank loss: 0.8041553497314453
rank loss: 0.7314722537994385
rank loss: 0.7255677580833435
rank loss: 0.7017753720283508
rank loss: 0.5449838638305664
rank loss: 0.5712422728538513
rank loss: 0.5553099513053894
rank loss: 0.6157011985778809
rank loss: 0.8042247295379639
rank loss: 0.

rank loss: 0.6184329986572266
rank loss: 0.5074203610420227
rank loss: 0.5009621977806091
rank loss: 0.6265016198158264
rank loss: 0.5759097933769226
rank loss: 0.5756366848945618
rank loss: 0.6621382236480713
rank loss: 0.6761197447776794
rank loss: 0.5883328914642334
rank loss: 0.5905451774597168
rank loss: 0.6056548357009888
rank loss: 0.5463549494743347
rank loss: 0.6404637098312378
rank loss: 0.7027940154075623
rank loss: 0.6647210717201233
rank loss: 0.5131840109825134
rank loss: 0.61524498462677
rank loss: 0.5861734747886658
rank loss: 0.5971730947494507
rank loss: 0.4795689284801483
rank loss: 0.5862950086593628
rank loss: 0.6167292594909668
rank loss: 0.6143447160720825
rank loss: 0.582001805305481
rank loss: 0.6951585412025452
rank loss: 0.603801965713501
rank loss: 0.7294945120811462
rank loss: 0.6180760264396667
rank loss: 0.5084499716758728
rank loss: 0.5336926579475403
rank loss: 0.5536542534828186
rank loss: 0.5721023678779602
rank loss: 0.5547928810119629
rank loss: 0.6

rank loss: 0.7561215758323669
rank loss: 0.5722865462303162
rank loss: 0.4913157820701599
rank loss: 0.558089017868042
rank loss: 0.6448110938072205
rank loss: 0.612916111946106
rank loss: 0.5131893157958984
rank loss: 0.5444574356079102
rank loss: 0.4998959004878998
rank loss: 0.5485565066337585
rank loss: 0.5516345500946045
rank loss: 0.6016167402267456
rank loss: 0.639536440372467
rank loss: 0.5804027318954468
rank loss: 0.5594169497489929
rank loss: 0.6146384477615356
rank loss: 0.4861469268798828
rank loss: 0.416134238243103
rank loss: 0.8312742710113525
rank loss: 0.6786713004112244
rank loss: 0.5881346464157104
rank loss: 0.6396169066429138
rank loss: 0.43064650893211365
rank loss: 0.5152058005332947
rank loss: 0.46038588881492615
rank loss: 0.6111962795257568
rank loss: 0.46416160464286804
rank loss: 0.727708101272583
rank loss: 0.593927264213562
rank loss: 0.6528255939483643
rank loss: 0.5448377728462219
rank loss: 0.6585985422134399
rank loss: 0.5267945528030396
rank loss: 0.

In [19]:
torch.save(select_net, "qcentric.model")
torch.save(rank_net, "deeprank.model")

In [20]:
select_net_e = torch.load(f='qcentric.model')
rank_net_e = torch.load(f='deeprank.model')

list_gen = ListGenerator(rel_file=Letor07Path+'/relation.test.fold%d.txt'%(letor_config['fold']),
                         config=letor_config)
map_v = 0.0
map_c = 0.0

with torch.no_grad():
    for X1, X1_len, X1_id, X2, X2_len, X2_id, Y, F in \
        list_gen.get_batch(data1=loader.query_data, data2=loader.doc_data):
        print(X1.shape, X2.shape, Y.shape)
        X1, X1_len, X2, X2_len, Y, F = to_device(X1, X1_len, X2, X2_len, Y, F, device=select_device)
        X1, X2, X1_len, X2_len, X2_pos = select_net_e(X1, X2, X1_len, X2_len, X1_id, X2_id)
        X2, X2_len = utils.data_adaptor(X2, X2_len, select_net, rank_net, letor_config)
        #print(X1.shape, X2.shape, Y.shape)
        pred = rank_net_e(X1, X2, X1_len, X2_len, X2_pos)
        map_o = utils.eval_MAP(pred.tolist(), Y.tolist())
        #print(pred.shape, Y.shape)
        map_v += map_o
        map_c += 1.0
    map_v /= map_c

print('[Test]', map_v)

[./data/letor/r5w/relation.test.fold1.txt]
	Instance size: 13652
List Instance Count: 336
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(50, 20) (50, 2000) (50,)
(51, 20) (51, 2000) (51,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)


(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
(40, 20) (40, 2000) (40,)
[Test] 0.5028438732210702
